In [1]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun


api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool.name

'wikipedia'

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

loader = WebBaseLoader("https://swimm.io/learn/what-is-documentation/what-is-documentation-how-it-can-transform-your-organization#:~:text=Documentation%20is%20written%20information%20that,guides%2C%20and%20online%20help%20resources.")
documentation = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(documents=documentation)
db = FAISS.from_documents(text_splitter, embedding=OpenAIEmbeddings())
retriever = db.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F87E77D5A0>)

In [3]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="Documen_retriever",
    description="Retrieves documents based on a query",
)

retriever_tool.name

'Documen_retriever'

In [4]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

'arxiv'

In [5]:
tools = [wiki_tool, arxiv_tool, retriever_tool]
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Machine learning\\Generative AI\\Langchain\\langchain\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=1000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='Documen_retriever', description='Retrieves documents based on a query', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F87EE0B5B0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vec

In [11]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.4)

In [7]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:
# Agents
from langchain.agents import create_openai_tools_agent, AgentExecutor
agent = create_openai_tools_agent(llm, tools, prompt)

In [9]:
# An agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c1867281

In [10]:
agent_executor.invoke({"input": "Tell me about documentation."})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Documentation`


Page: Documentation
Summary: Documentation is any communicable material that is used to describe, explain or instruct regarding some attributes of an object, system or procedure, such as its parts, assembly, installation, maintenance, and use. As a form of knowledge management and knowledge organization, documentation can be provided on paper, online, or on digital or analog media, such as audio tape or CDs. Examples are user guides, white papers, online help, and quick-reference guides. Paper or hard-copy documentation has become less common. Documentation is often distributed via websites, software products, and other online applications.
Documentation as a set of instructional materials shouldn't be confused with documentation science, the study of the recording and retrieval of information.Documentation refers to any communicable material used to describe, explain, or instruct regarding the attrib

{'input': 'Tell me about documentation.',
 'output': "Documentation refers to any communicable material used to describe, explain, or instruct regarding the attributes of an object, system, or procedure. This includes details about its parts, assembly, installation, maintenance, and use. It serves as a form of knowledge management and organization and can be provided in various formats such as:\n\n- Paper\n- Online\n- Digital media (e.g., audio tapes, CDs)\n\nExamples of documentation include:\n\n- User guides\n- White papers\n- Online help\n- Quick-reference guides\n\nWhile paper or hard-copy documentation has become less common, documentation is often distributed via websites, software products, and other online applications.\n\nIt's important to note that documentation as a set of instructional materials is different from documentation science, which is the study of recording and retrieving information."}